In [8]:
# Sample user submission
message = '''
    Create a count for each character in the string.
    For each character:
    Convert it to lowercase.
    Count how many times it appears.
    Set total_length to 0 and odd_count to 0.
    For each character count:
    Add the even part of the count to total_length.
    If the count is odd, add 1 to odd_count.
    If odd_count is greater than 0, add 1 to total_length.
    Return total_length.
'''


In [18]:
import httpx
import os
from dotenv import load_dotenv

load_dotenv()

def find_problem(problem_id: int):
    api_url = f'{os.getenv("DOCKER_API_BASE_URL")}/problems/{problem_id}' if os.path.exists('/.dockerenv') else f'{os.getenv("API_BASE_URL")}/problems/{problem_id}'
    headers = { 'Content-Type': 'application/json' }
    print(f'🔍 Sending GET request to {api_url}...')
    try:
        print(f'api_url: {api_url}, headers: {headers}')
        response = httpx.get(api_url, headers=headers)
        if 200 <= response.status_code < 300:
            print(f'✅ Successfully found problem with id {problem_id}')
            return response.json()
        else:
            print(f'❌ Failed to get problem with status code {response.status_code}')

    except httpx.RequestError as e:
        print(f'🚨 Error sending request: {e}')

problem_data = find_problem(1)

# Define prompt
prompt = f'''
    You are a professor who specializes in solving code problems.
    The user will submit in words their approach to solving the problem.
    Analyze the user's submission and provide feedback.

    Output your feedback in JSON format with the following structure:
    {{
        "title": "Create a creative title of the analysis",
        "analysis": "Create a detailed analysis on the user's submission in a list of strings",
        "suggestions": ["Create suggestions for the user to improve their method in a list of strings"],
        "score": "An integer score: 1 (incorrect), 2 (partially correct), 3 (correct)"
    }}

    Problem Details:
    Difficulty: {problem_data['difficulty']}
    Title: {problem_data['title']}
    Description: 
    {problem_data['description']}
    Constraints: 
    {''.join([f'{constraint}' for constraint in problem_data['constraints']])}
    Examples: 
    {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}

    Here is the user's submission: {message}

    Ensure your output only contains the JSON object without any additional text.
    Provide feedback directly addressing the user as ‘you’ instead of referring to them in the third person.
'''

print(prompt)


🔍 Sending GET request to http://localhost:80/problems/1...
api_url: http://localhost:80/problems/1, headers: {'Content-Type': 'application/json'}
✅ Successfully found problem with id 1

    You are a professor who specializes in solving code problems.
    The user will submit in words their approach to solving the problem.
    Analyze the user's submission and provide feedback.

    Output your feedback in JSON format with the following structure:
    {
        "title": "Create a creative title of the analysis",
        "analysis": "Create a detailed analysis on the user's submission in a list of strings",
        "suggestions": ["Create suggestions for the user to improve their method in a list of strings"],
        "score": "An integer score: 1 (incorrect), 2 (partially correct), 3 (correct)"
    }

    Problem Details:
    Difficulty: 1
    Title: Climbing Stairs
    Description: 
    You are climbing a staircase. It takes n steps to reach the top.

Each time you can either climb 1 

In [ ]:
from ollama import Client

# Must have ollama on in the background
def ask_model(prompt: str):
    try:
        response = Client().generate(model='deepseek-r1:7b', prompt=prompt)
        print(response['response'])
    except Exception as e:
        print(f'🚨 Error: {e}')

ask_model(prompt)
